## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#Adding special considerations to drop list as it only applies to 27 of 30,000+ events
application_df.drop(['EIN', 'NAME', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'], axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,6692,1
4,T3,Independent,C1000,Heathcare,Trust,142590,1


In [3]:
# Determine the number of unique values in each column.
application_cats = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[application_cats].nunique()

#APPLICATION_TYPE AND CLASSIFICATION HAVE MORE THAN 10 UNIQUE VALUES

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_count = application_df.APPLICATION_TYPE.value_counts()
application_type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cut_off_amt = 500
application_types_to_replace = application_type_count[application_type_count <= cut_off_amt].index.tolist()


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classifcations_count = application_df.CLASSIFICATION.value_counts()
classifcations_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classifcations_count[classifcations_count>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cut_off_amt = 1800
classifications_to_replace = classifcations_count[classifcations_count <= cut_off_amt].index.tolist()


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_data_numeric = pd.get_dummies(application_df,dtype=float)
categorical_data_numeric.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = categorical_data_numeric['IS_SUCCESSFUL'].values
X = categorical_data_numeric.drop('IS_SUCCESSFUL', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 2000
hidden_nodes_layer2 = 1000
hidden_nodes_layer3 = 500
hidden_nodes_layer4 = 250
hidden_nodes_layer5 = 125

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2000)              64000     
                                                                 
 dense_1 (Dense)             (None, 1000)              2001000   
                                                                 
 dense_2 (Dense)             (None, 500)               500500    
                                                                 
 dense_3 (Dense)             (None, 250)               125250    
                                                                 
 dense_4 (Dense)             (None, 125)               31375     
                                                                 
 dense_5 (Dense)             (None, 1)                 126       
                                                                 
Total params: 2,722,251
Trainable params: 2,722,251
Non-

In [13]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
nn.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

#nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,batch_size=200, epochs=100)

Epoch 1/100
129/129 [==============================] - 29s 199ms/step - loss: 0.5759 - accuracy: 0.7204
Epoch 2/100
129/129 [==============================] - 23s 179ms/step - loss: 0.5654 - accuracy: 0.7259
Epoch 3/100
129/129 [==============================] - 13s 105ms/step - loss: 0.5646 - accuracy: 0.7274
Epoch 4/100
129/129 [==============================] - 14s 105ms/step - loss: 0.5615 - accuracy: 0.7271
Epoch 5/100
129/129 [==============================] - 14s 105ms/step - loss: 0.5603 - accuracy: 0.7282
Epoch 6/100
129/129 [==============================] - 14s 107ms/step - loss: 0.5595 - accuracy: 0.7285
Epoch 7/100
129/129 [==============================] - 15s 113ms/step - loss: 0.5602 - accuracy: 0.7297
Epoch 8/100
129/129 [==============================] - 14s 105ms/step - loss: 0.5598 - accuracy: 0.7284
Epoch 9/100
129/129 [==============================] - 15s 115ms/step - loss: 0.5581 - accuracy: 0.7296
Epoch 10/100
129/129 [==============================] - 14s 106m

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 2s - loss: 0.5568 - accuracy: 0.7293 - 2s/epoch - 8ms/step
Loss: 0.5567858815193176, Accuracy: 0.7293294668197632


In [33]:
from google.colab import drive
drive.mount('/content/drive')

# Export our model to HDF5 file
nn.save('./drive/My Drive/Export/AlphabetSoupCharity_Optimization_checkpoint.h5')

#https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_load.ipynb
#it is not saving the h5 document

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Attempt #1 to optimize - less hidden layers


In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 250
hidden_nodes_layer2 = 250
hidden_nodes_layer3 = 250
hidden_nodes_layer4 = 250
#hidden_nodes_layer5 = 250

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Fifth hidden layer -  not used this time

#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 250)               8000      
                                                                 
 dense_7 (Dense)             (None, 250)               62750     
                                                                 
 dense_8 (Dense)             (None, 250)               62750     
                                                                 
 dense_9 (Dense)             (None, 250)               62750     
                                                                 
 dense_10 (Dense)            (None, 1)                 251       
                                                                 
Total params: 196,501
Trainable params: 196,501
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
nn.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [19]:
fit_model = nn.fit(X_train_scaled,y_train,batch_size=200, epochs=100)

Epoch 1/100
129/129 [==============================] - 4s 17ms/step - loss: 0.5746 - accuracy: 0.7199
Epoch 2/100
129/129 [==============================] - 1s 11ms/step - loss: 0.5641 - accuracy: 0.7244
Epoch 3/100
129/129 [==============================] - 1s 11ms/step - loss: 0.5616 - accuracy: 0.7268
Epoch 4/100
129/129 [==============================] - 1s 11ms/step - loss: 0.5608 - accuracy: 0.7276
Epoch 5/100
129/129 [==============================] - 1s 11ms/step - loss: 0.5593 - accuracy: 0.7290
Epoch 6/100
129/129 [==============================] - 1s 10ms/step - loss: 0.5581 - accuracy: 0.7287
Epoch 7/100
129/129 [==============================] - 1s 11ms/step - loss: 0.5579 - accuracy: 0.7294
Epoch 8/100
129/129 [==============================] - 1s 10ms/step - loss: 0.5571 - accuracy: 0.7289
Epoch 9/100
129/129 [==============================] - 2s 15ms/step - loss: 0.5571 - accuracy: 0.7290
Epoch 10/100
129/129 [==============================] - 2s 15ms/step - loss: 0.556

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5587 - accuracy: 0.7273 - 663ms/epoch - 2ms/step
Loss: 0.5587236285209656, Accuracy: 0.7273469567298889


In [21]:
nn.save('./drive/My Drive/Export/AlphabetSoupCharity_Optimization_Trial1.h5')

Attempt 2 - smaller batch and 5 hidden

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 250
hidden_nodes_layer2 = 250
hidden_nodes_layer3 = 250
hidden_nodes_layer4 = 250
hidden_nodes_layer5 = 250

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Fifth hidden layer -  not used this time
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 250)               8000      
                                                                 
 dense_12 (Dense)            (None, 250)               62750     
                                                                 
 dense_13 (Dense)            (None, 250)               62750     
                                                                 
 dense_14 (Dense)            (None, 250)               62750     
                                                                 
 dense_15 (Dense)            (None, 250)               62750     
                                                                 
 dense_16 (Dense)            (None, 1)                 251       
                                                                 
Total params: 259,251
Trainable params: 259,251
Non-tr

In [23]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
nn.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [24]:
fit_model = nn.fit(X_train_scaled,y_train,batch_size=50, epochs=100)

Epoch 1/100
515/515 [==============================] - 7s 12ms/step - loss: 0.5761 - accuracy: 0.7217
Epoch 2/100
515/515 [==============================] - 5s 9ms/step - loss: 0.5648 - accuracy: 0.7256
Epoch 3/100
515/515 [==============================] - 5s 9ms/step - loss: 0.5628 - accuracy: 0.7260
Epoch 4/100
515/515 [==============================] - 6s 11ms/step - loss: 0.5609 - accuracy: 0.7269
Epoch 5/100
515/515 [==============================] - 4s 9ms/step - loss: 0.5596 - accuracy: 0.7277
Epoch 6/100
515/515 [==============================] - 6s 11ms/step - loss: 0.5598 - accuracy: 0.7292
Epoch 7/100
515/515 [==============================] - 4s 8ms/step - loss: 0.5586 - accuracy: 0.7269
Epoch 8/100
515/515 [==============================] - 4s 9ms/step - loss: 0.5577 - accuracy: 0.7286
Epoch 9/100
515/515 [==============================] - 6s 11ms/step - loss: 0.5573 - accuracy: 0.7286
Epoch 10/100
515/515 [==============================] - 4s 8ms/step - loss: 0.5570 - ac

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5605 - accuracy: 0.7287 - 708ms/epoch - 3ms/step
Loss: 0.560465931892395, Accuracy: 0.7287463545799255


In [26]:
nn.save('./drive/My Drive/Export/AlphabetSoupCharity_Optimization_Trial2.h5')

Trial 3 - sigmoid instead of relu

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 250
hidden_nodes_layer2 = 250
hidden_nodes_layer3 = 250
hidden_nodes_layer4 = 250
hidden_nodes_layer5 = 250

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Fifth hidden layer -  not used this time
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 250)               8000      
                                                                 
 dense_18 (Dense)            (None, 250)               62750     
                                                                 
 dense_19 (Dense)            (None, 250)               62750     
                                                                 
 dense_20 (Dense)            (None, 250)               62750     
                                                                 
 dense_21 (Dense)            (None, 250)               62750     
                                                                 
 dense_22 (Dense)            (None, 1)                 251       
                                                                 
Total params: 259,251
Trainable params: 259,251
Non-tr

In [28]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
nn.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [29]:
fit_model = nn.fit(X_train_scaled,y_train,batch_size=50, epochs=100)

Epoch 1/100
515/515 [==============================] - 7s 11ms/step - loss: 0.6143 - accuracy: 0.6819
Epoch 2/100
515/515 [==============================] - 4s 8ms/step - loss: 0.5865 - accuracy: 0.7247
Epoch 3/100
515/515 [==============================] - 4s 8ms/step - loss: 0.5839 - accuracy: 0.7269
Epoch 4/100
515/515 [==============================] - 6s 12ms/step - loss: 0.5791 - accuracy: 0.7266
Epoch 5/100
515/515 [==============================] - 4s 8ms/step - loss: 0.5689 - accuracy: 0.7258
Epoch 6/100
515/515 [==============================] - 5s 11ms/step - loss: 0.5661 - accuracy: 0.7262
Epoch 7/100
515/515 [==============================] - 5s 9ms/step - loss: 0.5656 - accuracy: 0.7262
Epoch 8/100
515/515 [==============================] - 6s 11ms/step - loss: 0.5654 - accuracy: 0.7268
Epoch 9/100
515/515 [==============================] - 6s 12ms/step - loss: 0.5633 - accuracy: 0.7258
Epoch 10/100
515/515 [==============================] - 4s 8ms/step - loss: 0.5634 - a

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5567 - accuracy: 0.7300 - 759ms/epoch - 3ms/step
Loss: 0.5567498803138733, Accuracy: 0.7300291657447815


In [37]:
nn.save('AlphabetSoupCharity_Optimization_Trial3.h5')

Additional trials used:

Dropping SPECIAL_CONSIDERATIONS and using:
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  500
hidden_nodes_layer2 = 250
hidden_nodes_layer3 =  125
hidden_nodes_layer4 = 65

resulted in 72.5947%

Not dropping anything and then Using:
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  500
hidden_nodes_layer2 = 250
hidden_nodes_layer3 =  125
hidden_nodes_layer4 = 65

resulted in 72.501456%

Batch 50, epoch 150 and number_input_features = len(X_train[0]) hidden_nodes_layer1 = 500 hidden_nodes_layer2 = 250 hidden_nodes_layer3 = 125 hidden_nodes_layer4 = 65

resulted in 72.61807%


Even adjusting the batch, epochs, and/or more hidden layers I cannot peak over 75% accuracy.

I tried removing status, removing special_conditions.  No help.

Tried smaller and larger batch sizes, more and less epochs.  

Tried changing relu to elu.

Nothing helps.  I am at a loss as to what can help break 75%.

Tried dropping MANY more columns, adjusted a longer epoch.  Still stuck in the 72-73% zone
